In [22]:
import numpy as np
import pandas as pd

In [23]:
def id(number):
    if len(str(number))==1:
        return '00'+str(number)
    else:
        return '0'+str(number)

In [24]:
foods = []
dexcoms = []
to_return = {}
for i in range(1, 17):
    current_id = id(i)
    foods.append(pd.read_csv(f'data/{current_id}/Food_Log_{current_id}.csv'))
    dexcoms.append(pd.read_csv(f'data/{current_id}/Dexcom_{current_id}.csv'))

In [25]:
for i in range(16):
    dexcom = dexcoms[i]
    food = foods[i]
    dexcom['time'] = pd.to_datetime(dexcom['Timestamp (YYYY-MM-DDThh:mm:ss)'])
    dexcom = dexcom[dexcom['time'].notna()]
    food['time'] = pd.to_datetime(food['time_begin'])
    food = food[food['time'].notna()]

    #food = food.groupby('time', as_index=False)[['calorie', 'total_carb', 'dietary_fiber', 'sugar', 'protein', 'total_fat']].sum()
    food = food.groupby('time', as_index=False).agg({
        'calorie': 'sum',
        'total_carb': 'sum',
        'dietary_fiber': 'sum',
        'sugar': 'sum',
        'protein': 'sum',
        'total_fat': 'sum',
        'logged_food': list,     # example column to aggregate as list
    })
    dexcom = dexcom.sort_values('time')
    food = food.sort_values('time')

    # Create a results dataframe to store our findings
    results = []
    for _, food_elem in food.iterrows():
        # Find the closest timestamp in df that comes before food_time
        food_time = food_elem['time']
        mask = dexcom['time'] <= food_time
        if not mask.any():
            continue  # Skip if no earlier timestamp exists
        
        closest_time = dexcom.loc[mask, 'time'].max()
        basis_glucose = dexcom.loc[dexcom['time'] == closest_time, 'Glucose Value (mg/dL)'].iloc[0]
    
        # Find all records within one hour after the closest_time
        one_hour_later = closest_time + pd.Timedelta(hours=1)
        time_window = dexcom[(dexcom['time'] >= closest_time) & (dexcom['time'] <= one_hour_later)]
    
        # Calculate sugar changes relative to the basis
        for _, row in time_window.iterrows():
            results.append({
                'food_time': food_time,
                'basis_time': closest_time,
                'current_time': row['time'],
                'basis_glucose': basis_glucose,
                'current_glucose': row['Glucose Value (mg/dL)'],
                'sugar_change': row['Glucose Value (mg/dL)'] - basis_glucose,
                'minutes_from_basis': (row['time'] - closest_time).total_seconds() / 60,
                'calorie': food_elem['calorie'],
                'total_carb': food_elem['total_carb'],
                #'dietary_fiber': food_elem['dietary_fiber'],
                'sugar': food_elem['sugar'],
                'protein': food_elem['protein'],
                #'total_fat': food_elem['total_fat'],
                'logged_food': food_elem['logged_food']
            })
    results_df = pd.DataFrame(results)
    to_return[id(i+1)] = results_df

In [26]:
for i in range(1, 17):
    if i==3:
        continue
    to_return[id(i)].to_csv(f'data/{id(i)}/summary{id(i)}.csv')